To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [2]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.feature_extraction.text import TfidfTransformer 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Polina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install langdetect


In [ ]:
!pip install gensim

In [5]:
CREDENTIALS = service_account.Credentials.from_service_account_info({})

In [6]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n','') for i in Corpus]
    return corpus


"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


# удаление ссылок
def clearTextFromLinks(text):
    text_without_http=[]
    for sentence in text:   
        sentence_without_http = list(filter(lambda word: 
                            not re.search(r'transcript:visitor:', word) and
                            not re.search(r'http://', word) and 
                            not re.search(r'https://', word), 
                     sentence))
        text_without_http.append(sentence_without_http)        
    return text_without_http


# удаление символов и цифр
def clearTextFromNotLetter(text):
    clear_text = []
    for sentence in text:
        clear_sentence=[]
        for word in sentence:
            clear_words = re.split(r'[\d\W\']+', word)
            for j in clear_words:
                if len(j) > 1:
                    clear_sentence.append(j)
        clear_text.append(clear_sentence)
    return clear_text

"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1
Output: [["word2"],["word3","word4"]]
"""  
# удаление мусорных элементов
def clearTextAfterGarbage(Corpus,Substr):
    clear_corpus = []
    for text in Corpus:
        prepared_text = []
        for i in range(0, len(text)):
            cur_word = text[i]
            if type(Substr) is list:
                res = cur_word in Substr
#                 else:
#                      res = re.search(Substr, cur_word)
            if res == False:
                prepared_text.append(cur_word)

        clear_corpus.append(prepared_text)
            
    clear_text=clearTextFromNotLetter(clearTextFromLinks(clear_corpus))
    return clear_text

# топ часто встречаемых слов в датасете
def TopWords(list):
    top_words=[]
    for x in list:
        for i in x:
            top_words.append(i)
    frequency = {}
    for word in top_words:
        count = frequency.get(word,0)
        frequency[word] = count + 1

    frequency_list = frequency.keys()
    ranked_words = []

    for words in frequency_list:
        ranked_words.append([words, frequency[words]])
    ranked_words.sort(key=lambda x: x[1], reverse=True)
    return ranked_words

"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec


# count repeated words
def calcRepetitionWords(words):
    repetitionsWords = {}
    for word in words:
        if not repetitionsWords.get(word):
            repetitionsWords[word] = 1
        else:
            repetitionsWords[word] += 1
    return repetitionsWords


# count overlapping words
def calcIntersectionsWithQuantityInWords(fWords, sWords):
    repetitionsFWords = calcRepetitionWords(fWords)
    repetitionsSWords = calcRepetitionWords(sWords)
    intersectionRepetitionsWords = {}
    for word in repetitionsFWords:
        if repetitionsSWords.get(word):
            intersectionRepetitionsWords[word] = repetitionsFWords[word] + repetitionsSWords[word]
    return intersectionRepetitionsWords

RAWDATA 

In [7]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

Downloading: 100%|███████████████████████████████████████████████████████████| 23450/23450 [00:20<00:00, 1140.79rows/s]


(23450, 4)

In [8]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag
0,\nChat transcript:\nVisitor: I want to buy wit...,chat,ps,0
1,\nChat transcript:\nVisitor: hell i had a prob...,chat,ps,0
2,\nChat transcript:\nVisitor: لا استطيع الشراء ...,chat,ps,0
3,\nChat transcript:\nVisitor: im having trouble...,chat,ps,0
4,\nChat transcript:\nVisitor: Hi\nAna: Hello. H...,chat,ps,0
5,\nChat transcript:\nVisitor: Здраствуйте\nAna:...,chat,ps,0
6,\nChat transcript:\nVisitor: hello\nVisitor: w...,chat,ps,0
7,\nChat transcript:\nVisitor: not letting me bu...,chat,ps,0
8,\nChat transcript:\nVisitor: مرحبا\nAna: Hello...,chat,ps,0
9,\nChat transcript:\nVisitor: hi im trying to b...,chat,ps,0


In [9]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


DATA PREPROC

In [10]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = lowerCase(corpus)

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [11]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [12]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [13]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag,lang
23440,"chat transcript:visitor: hello, i bought pubg ...",chat,other,1,en
23441,chat transcript:visitor: good dayi accidentall...,facebook,other,1,en
23442,chat transcript:visitor: payment not going thr...,chat,other,1,en
23443,chat transcript:visitor: paid for faceit subsc...,chat,other,1,en
23444,"chat transcript:visitor: hello, i am trying to...",chat,other,1,en
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en
23449,chat transcript:visitor: hivisitor: hellovisit...,chat,other,1,en


In [14]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en


In [15]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [16]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript:visitor:, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript:visitor:, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript:visitor:, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript:visitor:, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript:visitor:, hiana:, hello., ho...",chat,ps,0,en
...,...,...,...,...,...
23445,"[chat, transcript:visitor:, hi,, i, made, a, p...",chat,other,1,en
23446,"[chat, transcript:visitor:, hi,, how, long, wi...",chat,other,1,en
23447,"[chat, transcript:visitor:, i, bought, playeru...",chat,other,1,en
23448,"[chat, transcript:visitor:, good, day, i, took...",chat,other,1,en


In [17]:
texts = list(SupportDataframe_eng.description)

In [18]:
#cleaning text after garbage
my_stopwords = stopwords.words('english')[:20] + ['chat']
tests_clear = clearTextAfterGarbage(texts.copy(), my_stopwords)

In [19]:
SupportDataframe_eng['description'] = tests_clear

In [20]:
category_ps = SupportDataframe_eng[SupportDataframe_eng['category_flag'] == 0]['description']
category_other = SupportDataframe_eng[SupportDataframe_eng['category_flag'] == 1]['description']
category_ps_list = sum(category_ps.tolist(), [])
category_other_list = sum(category_other.tolist(), [])

In [21]:
# выполняется оверсемплинг, увеличение набора с category_flag = 0
oversampled_data = pd.concat([SupportDataframe_eng[SupportDataframe_eng['category_flag']==1] ,
                              SupportDataframe_eng[SupportDataframe_eng['category_flag']==0].sample(len(category_other), 
                                                                                                    replace=True) ])

TRAIN TEST SPLIT DATAFRAMES

In [22]:
#list of unique categories
unique_categories = np.unique(oversampled_data.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [23]:
descriptions = oversampled_data['description']
categories = oversampled_data['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [24]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(XTrain)

In [25]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(11370694, 16272265)

In [26]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [ ]:
TrainVecs

In [27]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(2865030, 4109085)

In [28]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [ ]:
TestVecs

CROSSVALIDATION AND BUILD CLASSIFIER

In [29]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [30]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.91646371 0.91914272 0.91646371 0.91548953 0.91790499]
0.917092932537453


In [31]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=40, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [32]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[2154  413]
 [ 644 1922]]
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      2567
           1       0.82      0.75      0.78      2566

    accuracy                           0.79      5133
   macro avg       0.80      0.79      0.79      5133
weighted avg       0.80      0.79      0.79      5133



USE StratifiedKFold

In [33]:
KFold=StratifiedKFold(n_splits=5, shuffle=False, random_state=42)
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = KFold)
print(Scores)
print(np.mean(Scores))

[0.91646371 0.91914272 0.91646371 0.91548953 0.91790499]
0.917092932537453


In [34]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=40, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [35]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[2154  413]
 [ 644 1922]]
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      2567
           1       0.82      0.75      0.78      2566

    accuracy                           0.79      5133
   macro avg       0.80      0.79      0.79      5133
weighted avg       0.80      0.79      0.79      5133



USE TF-IDF

In [36]:
new_XTrain=[' '.join(i) for i in XTrain]
new_XTest=[' '.join(i) for i in XTest]

In [37]:
text_clf=TfidfVectorizer()
tfidf = text_clf.fit_transform(new_XTrain)
MLP = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40).fit(tfidf,YTrain)
idf =  MLP.predict(text_clf.transform(new_XTest))

In [38]:
print(confusion_matrix(YTest,idf))
print(classification_report(YTest,idf))

[[2550   17]
 [ 216 2350]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      2567
           1       0.99      0.92      0.95      2566

    accuracy                           0.95      5133
   macro avg       0.96      0.95      0.95      5133
weighted avg       0.96      0.95      0.95      5133

